<div class="alert alert-info">
    <h2 align="center"> A new profiling approach for DNA sequences based on the nucleotides' physicochemical features: Corona case study</h2>
    <h3 align="center"><a href="http://sharif.edu/~koohi/">Saeedeh Akbari Rokn Abadi</a>, <a href="http://sharif.edu/~koohi/">Amirhossein Mohammadi</a>, <a href="http://sharif.edu/~koohi/">Somayyeh Koohi</a></h3>
</div>

## Import Library and Modules


In [15]:
import time
import os, os.path
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd 
import sys
import re
import random
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score,balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import warnings
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

## Define Encoding Method

In [2]:
def Change_DNA(dna):
    line = "".join(dna.split("\n")[1:])
    dna = line.upper()
    return dna

def PC_mer(dna):    
    vector = 1024 #Determining the length of three vectors {weak or strong}, {amino or keto} and {purine or pyrimidine} based on k-mer size==>2^k
    start = vector // 2
    x = start

    weakStrong = [0] * (vector) #CG , AT
    aminoKeto = [0] * (vector) #AC , GT
    PurPyr = [0] * (vector) #AG , CT

    #weak or strong
    for i in range(len(dna)):
        if dna[i] == "C" or dna[i] == "G":
            x = (x + vector) // 2
            weakStrong[x] = weakStrong[x] + 1 
        
        elif dna[i] == "A" or dna[i] == "T":
            x = (x + 0) // 2
            weakStrong[x] = weakStrong[x] + 1

    x = start
    #amino or keto       
    for i in range(len(dna)):
        if dna[i] == "C" or dna[i] == "A":
            x = (x + vector) // 2
            aminoKeto[x] = aminoKeto[x] + 1 
        
        elif dna[i] == "G" or dna[i] == "T":
            x = (x + 0) // 2
            aminoKeto[x] = aminoKeto[x] + 1

    x = start
    #purine or pyrimidine
    for i in range(len(dna)):
        if dna[i] == "C" or dna[i] == "T":
            x = (x + vector) // 2
            PurPyr[x] = PurPyr[x] + 1 
        
        elif dna[i] == "A" or dna[i] == "G":
            x = (x + 0) // 2
            PurPyr[x] = PurPyr[x] + 1

    arr = np.concatenate((PurPyr, aminoKeto, weakStrong))
    return arr

## Reading Data From the Specified Path and Encode Them Based on PC-mer Coding and Extracting Labels

In [4]:
ffolders = []
dna = []
label = []
# define Dataset path
mypath = 'C://Users//asus//Desktop/github/Human Coronavirus/'
my_list = os.listdir(mypath)

            
for i in range(len(my_list)):
    path = mypath + "\\" + my_list[i]
    s = os.listdir(path)
    n = len(s)
    
    for j in range(n):
        path1 = path + "\\" + s[j]
        f = open(path1 , "r")
        line = f.read()
        f.close()
        line = Change_DNA(line)
        x = PC_mer(line)
        dna.append(x)
        label.append(i)            

## Create A Data Frame (Label & Extracted vector based on PC-mer Coding)

In [5]:
df = pd.DataFrame({  
    "label": label,
    "dna": dna,
})
df

,label,dna
0,0,"[37, 14, 14, 18, 18, 22, 19, 18, 14, 24, 20, 2..."
1,0,"[20, 15, 16, 15, 21, 22, 15, 20, 13, 25, 16, 2..."
2,0,"[6, 13, 12, 18, 16, 20, 19, 17, 13, 23, 19, 28..."
3,0,"[8, 14, 13, 18, 17, 20, 19, 17, 14, 23, 19, 28..."
4,0,"[12, 15, 17, 18, 25, 18, 16, 22, 20, 23, 15, 2..."
...,...,...
869,6,"[32, 31, 33, 45, 29, 35, 28, 48, 24, 29, 25, 3..."
870,6,"[29, 25, 30, 40, 27, 34, 27, 42, 22, 25, 24, 3..."
871,6,"[32, 31, 33, 45, 30, 35, 28, 48, 24, 30, 25, 3..."
872,6,"[32, 31, 33, 45, 29, 35, 28, 48, 24, 29, 25, 3..."


## Save the ٍٍٍExtracted Vectors

In [6]:
df.to_csv('Human Coronavirus.csv',index=False)

## Define Default parameters

In [18]:
X = list(df['dna'])
y = list(df['label'])
X=np.array(X)
y=np.array(y)
kfold = 10
sumSen = 0
sumSpec = 0
sumP = 0
sumR = 0
sumF = 0
sumacc=0
sumaccbl=0
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'CM')
if not os.path.exists(final_directory):
    os.makedirs(final_directory)

## Define Metrics Methods

In [19]:
def get_metrics(y_test, y_predicted):
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    acc=accuracy_score(y_test, y_predicted)
    return precision, recall, f1, acc


def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1   
    sen = TP / (TP+FN)
    spec = TN / (TN+FP)
    return(sen , spec)

def comp_confmat(actual, predicted):
    classes = np.unique(actual) # extract the different classes
    matrix = np.zeros((len(classes), len(classes))) # initialize the confusion matrix with zeros
    for i in range(len(classes)):
        for j in range(len(classes)):
            matrix[i, j] = np.sum((actual == classes[i]) & (predicted == classes[j]))
    return matrix

## StratifiedKFold

In [20]:
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=20)

# Define Model

In [21]:
from sklearn.svm import SVC
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import GaussianNB
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.neural_network import MLPClassifier
#from sklearn.neighbors import NearestCentroid

## Train Method

In [22]:
t1 = time.time()
tempfold=1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = make_pipeline(StandardScaler(), SVC(kernel='linear')).fit(X_train, y_train)
#    clf = LogisticRegression().fit(X_train, y_train)
#     clf = GaussianNB().fit(X_train, y_train)
#     clf = LinearDiscriminantAnalysis().fit(X_train, y_train)
#     clf = DecisionTreeClassifier().fit(X_train, y_train)
#     clf = MLPClassifier().fit(X_train, y_train)
#     clf = NearestCentroid().fit(X_train, y_train)
#     clf = NearestCentroid(metric='manhattan').fit(X_train, y_train)
    
    
    y_pred = clf.predict(X_test)
    cm=comp_confmat(y_test,y_pred)
    np.savetxt('CM/'+'CM_fold'+str(tempfold)+'.txt',cm)
    f = open("Report.txt", "a")
    f.write("Fold"+str(tempfold)+':\n')
    precision, recall, f1,acc = get_metrics(y_test, y_pred)
    f.write("precision:"+str(precision)+'\n')
    f.write("recall:"+str(recall)+'\n')
    f.write("f1:"+str(f1)+'\n')
    f.write("acc:"+str(acc)+'\n')
    sumP = sumP + precision
    sumR = sumR + recall
    sumF = sumF + f1
    sumacc=sumacc+acc
    f.close()
    tempfold=tempfold+1
    
finalPrec = sumP/kfold
finalRcl = sumR/kfold
finalF = sumF/kfold
finalacc=sumacc/kfold
t2 = time.time()
totalTime = t2 - t1
print("time = %.2f"  %(totalTime))
print("accuracy = %.3f" %(finalacc))
print("precision = %.3f \nrecall = %.3f \nf1 = %.3f" %(finalPrec, finalRcl, finalF))

time = 3.17
accuracy = 1.000
precision = 1.000 
recall = 1.000 
f1 = 1.000


## Save Info

In [23]:
f = open("Metric.txt", "a")
f.write("Total time:"+str(totalTime)+'\n')
f.write("Sensitivity:"+str(finalSen)+'\n')
f.write("Specificity:"+str(finalSpec)+'\n')
f.write("precision:"+str(finalPrec)+'\n')
f.write("recall:"+str(finalRcl)+'\n')
f.write("f1:"+str(finalF)+'\n')
f.write("accuracy:"+str(finalacc)+'\n')
f.close()